In [1]:
import numpy as np
from functools import reduce

In [3]:
def kron_all(*gates):
    # Flattens nested gate lists like ([Gate, Gate], Gate)
    flat = []
    for g in gates:
        if isinstance(g, list):
            flat += g
        else:
            flat.append(g)
    return reduce(lambda a, b: Gate(np.kron(a.matrix, b.matrix)), flat)

def gate_pow(gate, n):
    return [gate] * n

In [301]:
class State:
    def __init__(self, vector):
        self.vector = vector

    def __repr__(self):
        return f"State(vector=\n{self.vector})"

    def operate(self, gate):
        return State(gate.matrix @ self.vector)

    def product(self, other):
        return State(np.kron(self.vector, other.vector))

    def project(self, other):
        return np.tensordot(self.vector, other.vector)

    def norm(self):
        return np.linalg.norm(self.vector)
        
    def normalise(self):
        return State(self.vector / self.norm())

    def outer(self, other):
        return np.outer(self.vector, other.vector)

    def length(self):
        return len(self.vector)

    def __mul__(self, scalar):
        return State(self.vector * scalar)

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

In [137]:
class Gate:
    def __init__(self, matrix):
        self.matrix = matrix

    def __repr__(self):
        return f"Gate(matrix=\n{self.matrix})"

    def __matmul__(self, other):
        if isinstance(other, Gate):
            return Gate(np.matmul(self.matrix, other.matrix))
        elif isinstance(other, State):
            return State(self.matrix @ other.vector)
        return NotImplemented

    def __add__(self, other):
        if isinstance(other, Gate):
            return Gate(self.matrix + other.matrix)
        return NotImplemented

    def product(self, other):
        return Gate(np.kron(self.matrix, other.matrix))
        
    @staticmethod
    def arb_phase(theta):
        return Gate(np.array([[1,0],[0, np.exp(1j*theta/2)]]))

In [ ]:
class measurement:
    def __init__(self, psi):
        self.psi = psi

    def partial_measurement(self, num_meas_qubit, index_meas_qubit):
        q = int(np.log2(len(self.psi))) #Number of qubits in state

In [473]:
#Gates
I = Gate(np.identity(2))
H = Gate(1/np.sqrt(2) * np.array([[1,1],[1,-1]]))
X = Gate(np.array([[0,1],[1,0]]))
Y = Gate(np.array([[0,-1j],[1j,0]]))
Z = Gate(np.array([[1,0],[0,-1]]))
T = Gate.arb_phase(np.pi/2)
R_y = Gate(1/np.sqrt(2) * np.array([[1,1],[1j,-1j]]))

#States
zero = State(np.array([[1],[0]]))
one = State(np.array([[0],[1]]))
plus = H @ zero
minus = H @ one
right = R_y @ zero
left  = R_y @ one

#Two qubit gates
CNOT = Gate.product(Gate(State.outer(zero,zero)), I) + Gate.product(Gate(State.outer(one,one)), X)
rCNOT = Gate.product(I, Gate(State.outer(zero,zero))) + Gate.product(X, Gate(State.outer(one,one)))
SWAP = CNOT @ rCNOT @ CNOT

#Setting up the Bell states
def Bell(x=0, z=0):
    # Step 1: Prepare the initial 2-qubit state
    if x == 0 and z == 0:
        psi_0 = zero.product(zero)
    elif x == 1 and z == 0:
        psi_0 = one.product(zero)
    elif x == 0 and z == 1:
        psi_0 = zero.product(one)
    elif x == 1 and z == 1:
        psi_0 = one.product(one)
    else:
        raise ValueError("x and z must be 0 or 1")

    # Step 2: Apply H ⊗ I
    psi_1 = H.product(I) @ psi_0

    # Step 3: Apply CNOT
    psi_2 = CNOT @ psi_1

    return psi_2

#Arbitrary control gates
def control_arb(num_qubit, gate, control, target):
    if type(target) != list:
        target = [target]
        
    c = control 
    m = num_qubit - c - 1
    t = abs(min(target)-control) - 1 
    k = m - len(target) - t
    
    passive = kron_all(
        gate_pow(I, c),
        [Gate(State.outer(zero, zero))],
        gate_pow(I, m)
    )
    
    active = kron_all(
        gate_pow(I, c),
        [Gate(State.outer(one, one))],
        gate_pow(I, t),
        [gate] * len(target),
        gate_pow(I, k)
    )
    return passive + active

#Measurement
rng = np.random.default_rng()
def measurement_multiple(psi, basis, n = 1, m = 0):
    q = int(np.log2(psi.length())) #Number of qubits in state

    if basis == 'z':
        basis_set = [zero, one]
    if basis == 'x':
        basis_set = [plus, minus]
    if basis == 'y':
        basis_set = [right, left]
    
    projector_0 = kron_all(
        gate_pow(I, m),
        [Gate(State.outer(basis_set[0], basis_set[0]))] * n,
        gate_pow(I, q - m - n)
    )
    
    projector_1 = kron_all(
        gate_pow(I, m),
        [Gate(State.outer(basis_set[1], basis_set[1]))] * n,
        gate_pow(I, q - m - n)
    )
    
    state_0 = projector_0 @ psi
    state_1 = projector_1 @ psi

    output = rng.choice((0,1), p=[state_0.norm() ** 2, state_1.norm() ** 2])
    if output:
        state = state_1.normalise()
    else:
        state = state_0.normalise()
    return output, state

In [507]:
#Teleportation scheme does not totally work
tele_qubit = zero
basis = 'z'

#Step 0: Setup qubits
psi_0 = tele_qubit.product(Bell(0,0))
#print(psi_0)

#Step 1: Apply CNOT with qubit 1 as control and 2 as target
psi_1 = control_arb(3, X, 0, 1) @ psi_0

#Step 2a: Measure second qubit
output_1, psi_2 = measurement_multiple(psi_1, basis, m = 2)

#Step 2B: Apply Hadamard to qubit 1
psi_3 = kron_all(H, gate_pow(I, 2)) @ psi_2

#Step 3a: Apply CNOT with 2 as control and 3 as target
psi_4 = control_arb(3, X, 1, 2) @ psi_3

#Step 3b: Measure qubit 1
output_2, psi_5 = measurement_multiple(psi_4, basis)

#Step 4: Apply CZ with 1 as control and 3 as target
psi_5 = control_arb(3, Z, 0, 2) @ psi_4

output_3, psi_6 = measurement_multiple(psi_5, basis)
print(output_3)

1
